#### 1. Scrape an online E-Commerce Site for Data

In [2]:
!Pip install mysql

In [3]:
import requests
from fp.fp import FreeProxy
from bs4 import BeautifulSoup as bs
import mysql.connector
from mysql.connector import Error
import pandas as pd

ModuleNotFoundError: No module named 'mysql'

In [4]:
proxy = FreeProxy(country_id=['US']).get()
pro = { "http" : proxy}
print(pro)

{'http': 'http://23.94.86.138:80'}


In [ ]:
head = {
"Accept":
"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
"Accept-Encoding": "gzip, deflate, br, zstd",
"Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8",
"Cache-Control": "max-age=0",
"Device-Memory": "8",
"Downlink": "10",
"Dpr": "1.25",
"Ect": "4g",
"Priority": "u=0, i",
"Rtt": "0",
"Sec-Ch-Device-Memory": "8",
"Sec-Ch-Dpr": "1.25",
"Sec-Ch-Ua": "\"Not/A)Brand\";v=\"8\", \"Chromium\";v=\"126\", \"Google Chrome\";v=\"126\"",
"Sec-Ch-Ua-Mobile": "?1",
"Sec-Ch-Ua-Platform": "\"Android\"",
"Sec-Ch-Ua-Platform-Version": "\"6.0\"",
"Sec-Ch-Viewport-Width": "777",
"Sec-Fetch-Dest": "document",
"Sec-Fetch-Mode": "navigate",
"Sec-Fetch-Site": "none",
"Sec-Fetch-User": "?1",
"Upgrade-Insecure-Requests": "1",
"User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Mobile Safari/537.36",
"Viewport-Width": "777"
}


In [ ]:
response=requests.get('https://www.amazon.in/s?bbn=1389401031&rh=n%3A1389401031%2Cp_89%3AOnePlus',headers=head,proxies=pro)
response.status_code

In [ ]:
if response.status_code == 200:
    print(' Success while Data Scrapping !!!!!! ')
    sortedup = bs(response.content,features="lxml")
    divs = soup.find_all('div')
    cards = soup.find_all('div',{'class':"s-result-item s-asin sg-col sg-col-12-of-12 s-widget-spacingsmall"})
    print("Data fecthed : ",len(cards))
    
    t,p,r = [],[],[]

    for card in cards:
        price = card.find('span',{'class':'a-price-whole'})
        if price is None:
            continue
        title = card.find('span',{'class':'a-size-small a-color-base a-text-normal'})
        rating = card.find('span',{'class':'a-icon-alt'})

        if rating:
            rating = rating.text
        else:
            rating = "New arrival"
        t.append(title.text)
        p.append(price.text)
        r.append(rating)
        
        df = pd.DataFrame({"Title":t,"Price":p,"Rating":r})
        print(df.head(5))
        
        username = 'root'
        password = ''
        host = 'localhost'
        port = '3306'
        database = ''
        
        try:
            connection = mysql.connector.connect(
            host=host,
            user=username,
            password=password,
            database=database,
            port=port)

            print("Connection : ",connection)
    
            if connection.is_connected():
                print(" Mysql Connected Sucessfully !!!!!!!")
                cursor = connection.cursor()
                sql = "DROP TABLE IF EXISTS products"
                cursor.execute(sql)
                connection.commit()
                cursor.execute("CREATE TABLE products (title TEXT, price TEXT, Rating TEXT)")
                for data in zip(df.values):
                    sql = "INSERT INTO products (title, price, rating) VALUES (%s, %s,%s)"
                    val = (data[0][0],data[0][1],data[0][2])
                    cursor.execute(sql, val)
                    connection.commit()
                print("SQL operations Success !!!")
        except Error as e:
            print(f"Error: {e}")
        finally:
            if connection.is_connected():
                cursor.close()
                connection.close()
            print("MYSQL Connection Closed !")
else:
    print(response.status_code)

#### 2.Scrape an online Social Media Site for Data. Use python to scrape information from Time of India.

In [5]:
import requests
from bs4 import BeautifulSoup

#url = 'https://timesofindia.indiatimes.com/india'
url = 'https://timesofindia.indiatimes.com/india/maharashtra'
#headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36(KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
try:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    headlines = soup.find_all('span', {'class': 'w_tle'})
    for headline in headlines:
        text = headline.text.strip()
        link = headline.find('a')['href']
        print("HeadLine: ",text)
        print("Link: https://timesofindia.indiatimes.com",link)
        print()
except Exception as e:
    print("Error:", e)

HeadLine:  Drunk driver mows down college professor in Maharashtra's Palghar district
Link: https://timesofindia.indiatimes.com /city/pune/drunk-driver-mows-down-college-professor-in-maharashtras-palghar-district/articleshow/112212432.cms

HeadLine:  Backwardness is exceptional: Panel to Bombay HC on Maratha survey
Link: https://timesofindia.indiatimes.com /city/mumbai/backwardness-is-exceptional-panel-to-bombay-hc-on-maratha-survey/articleshow/112212423.cms

HeadLine:  Why this helplessness? HC raps Maharashtra for failure to act against illegal hawkers
Link: https://timesofindia.indiatimes.com /city/mumbai/why-this-helplessness-hc-raps-maharashtra-for-failure-to-act-against-illegal-hawkers/articleshow/112211691.cms

HeadLine:  'Identification of slum dwellers involves complicated process': Supreme Court on functioning of Maharashtra slum areas
Link: https://timesofindia.indiatimes.com /city/delhi/identification-of-slum-dwellers-involves-complicated-process-supreme-court-on-functionin

#### 3. Page Rank for link analysis using python Create a small set of pages namely page1, page2, page3 and page4 apply random walk on the same

In [6]:
import networkx as nx

G = nx.DiGraph()
G.add_nodes_from(['page1', 'page2', 'page3', 'page4'])
G.add_edges_from([('page1', 'page2'), ('page1', 'page2'), ('page1', 'page3'), ('page3', 'page1')])

pr = nx.pagerank(G, alpha=0.85)
rw = nx.pagerank(G)

print(rw)
print("the random walk is as follows")
print("page 1",rw['page1'])
print("page 2",rw['page2'])
print("page 3",rw['page3'])
print("page 4",rw['page4'])

print("the page ranks are as follows")
print("page 1",pr['page1'])
print("page 2",pr['page2'])
print("page 3",pr['page3'])
print("page 4",pr['page4'])

{'page1': 0.3465224265369786, 'page2': 0.26691678019989895, 'page3': 0.26691678019989895, 'page4': 0.1196440130632237}
the random walk is as follows
page 1 0.3465224265369786
page 2 0.26691678019989895
page 3 0.26691678019989895
page 4 0.1196440130632237
the page ranks are as follows
page 1 0.3465224265369786
page 2 0.26691678019989895
page 3 0.26691678019989895
page 4 0.1196440130632237


#### 4.Perform Spam Classifier

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
# Load the spam dataset
spam_df = pd.read_csv('spam_data.csv', encoding='latin-1')
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(spam_df['text'], spam_df['label'], test_size=0.2,
random_state=42)
# Vectorize the text data using a Bag of Words model
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)
# Train a Multinomial Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train_vect, y_train)
# Evaluate the accuracy of the model on the test set
accuracy = clf.score(X_test_vect, y_test)
print("Accuracy: {:.2f}%".format(accuracy*100))
# Test the model on a new message
#You have won a prize! Click here to claim it now!
new_message = [input("Enter your text data\n")]
new_message_vect = vectorizer.transform(new_message)
prediction = clf.predict(new_message_vect)
if(prediction[0]=="spam"):
    print("Your message is a spam")
else:
    print("Your message is genuine")

FileNotFoundError: [Errno 2] No such file or directory: 'spam_data.csv'

#### 5. Demonstrate Text Mining and Webpage Pre-processing using meta information from the web pages (Local/Online).

In [8]:
from bs4 import BeautifulSoup
import requests
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
# Download stopwords and initialize stemmer
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
# Retrieve webpage content
url = 'https://www.yahoo.com/?guccounter=1'
response = requests.get(url)
# Parse HTML and extract meta tags
soup = BeautifulSoup(response.content, 'html.parser')
meta_tags = soup.find_all('meta')

In [9]:
title = ''
description = ''
keywords = []
for tag in meta_tags:
    if tag.get('property')=='og:title':
        title=tag.get('content')
    if tag.get('property')=='og:description':
        discription=tag.get('content')
    if tag.get('name')=='keywords':
        keywords=tag.get('content')
# Preprocess content
content = soup.get_text()
tokens = word_tokenize(content)
filtered_tokens = [token.lower() for token in tokens if token.lower() not in stop_words]
stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
preprocessed_content = ' '.join(stemmed_tokens)
# Print results
print('Title:', title,'\n')
print('Description:', discription,'\n')
print('keywords:', keywords,'\n')
print('Preprocessed content:', preprocessed_content[0:100],'/n')

Title: Yahoo | Mail, Weather, Search, Politics, News, Finance, Sports & Videos 

Description: Latest news coverage, email, free stock quotes, live scores and video are just the beginning. Discover more every day at Yahoo! 

keywords: yahoo, yahoo home page, yahoo homepage, yahoo search, yahoo mail, yahoo messenger, yahoo games, news, finance, sport, entertainment 

Preprocessed content: yahoo | mail , weather , search , polit , news , financ , sport & video news today 's news us polit  /n


#### 6.Apriori Algorithm implementation in case study.

In [10]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
# Example dataset
dataset = [['bread', 'milk', 'eggs'],
['bread', 'diapers', 'beer', 'eggs'],
['milk', 'diapers', 'beer', 'cola'],
['bread', 'milk', 'diapers', 'beer', 'cola'],
['bread', 'milk', 'diapers', 'beer']]
# Convert the dataset to a one-hot encoded matrix
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
# Apply the Apriori algorithm to generate frequent itemsets
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
# Print the frequent itemsets
print(frequent_itemsets)

ModuleNotFoundError: No module named 'mlxtend'

In [ ]:
!pip install mlxtend

#### 7. Develop a basic crawler for the web search for user defined keywords.

In [ ]:
import requests
from bs4 import BeautifulSoup

def web_crawler(url, keywords):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        page_text = soup.get_text()
        # Check if any of the keywords are present on the page
        for keyword in keywords:
            if keyword.lower() in page_text.lower():
                print(f'{keyword} found on {url}')
            else:
                print(f'{keyword} not found on {url}')
    else:
        print(f'Request failed with status code {response.status_code}')
        
# Example usage
url = input("Enter the URL to be searched: ") #https://en.wikipedia.org/wiki/Web_crawler
keywords = []
print("Enter the keywords to be searched")
while True:
    k = input("Enter the keyword: ")
    keywords.append(k)
    x = int(input("Enter 1 to give more keyword, enter 0 to exit: "))
    if x == 0:
        break
web_crawler(url, keywords)

#### 8. Develop a programme for deep search implementation to detect plagiarism in documents online.


In [ ]:
import requests
from bs4 import BeautifulSoup
import difflib
import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
# Collect Data from User
text = input("Enter Text to Check for Plagiarism: ")
# Text Preprocessing
stop_words = set(nltk.corpus.stopwords.words('english'))
tokens = nltk.word_tokenize(text)
tokens = [token.lower() for token in tokens if token.isalpha()]
tokens = [token for token in tokens if token not in stop_words]
# Scrape Web for Similar Text
url = 'https://en.wikipedia.org/wiki/Python_(programming_language)'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
soup_text = soup.get_text()
soup_tokens = nltk.word_tokenize(soup_text)
soup_tokens = [token.lower() for token in soup_tokens if token.isalpha()]
soup_tokens = [token for token in soup_tokens if token not in stop_words]
# Calculate Similarity
similarity = difflib.SequenceMatcher(None, tokens, soup_tokens).ratio()
# Set Threshold for Plagiarism
threshold = 0.002
print(similarity)
# Plagiarism Detection
if similarity >= threshold:
    print("Text is Plagiarized.")
else:
    print("Text is Not Plagiarized.")

#### 9.  Sentiment analysis for reviews by customers and visualize the same

In [ ]:
import pandas as pd
from textblob import TextBlob
# read in the customer_reviews.csv file as a Pandas dataframe
reviews_df = pd.read_csv('customer_reviews.csv')
# create a new column in the dataframe to hold the sentiment polarity score for each review
reviews_df['sentiment_score'] = reviews_df['review_text'].apply(lambda x:
TextBlob(x).sentiment.polarity)
# categorize the sentiment scores into positive, negative, and neutral
reviews_df['sentiment_category'] = reviews_df['sentiment_score'].apply(lambda x: 'positive' if x > 0
else 'negative' if x < 0 else 'neutral')
# print out the count of reviews in each sentiment category for each product
for product_id in reviews_df['product_id'].unique():
    product_reviews_df = reviews_df[reviews_df['product_id'] == product_id]
    print('Product', product_id)
    print('Positive reviews:', len(product_reviews_df[product_reviews_df['sentiment_category'] =='positive']))
    print('Negative reviews:', len(product_reviews_df[product_reviews_df['sentiment_category'] =='negative']))
    print('Neutral reviews:', len(product_reviews_df[product_reviews_df['sentiment_category'] =='neutral']))
    print()

In [ ]:
!pip install textblob